## Lista de 250 melhores filmes

### Informações do Filme

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

driver = webdriver.Chrome()
driver.get('https://www.imdb.com/chart/top/')

lista_filmes = []

try:
    time.sleep(5)
    urls = driver.find_elements(By.CSS_SELECTOR, 'ul > li > div > div > div > div > a')
    url_filmes = [filme.get_attribute('href') for filme in urls]

    for url in url_filmes:
        driver.get(url)
        time.sleep(5)
        try:
            # Título Traduzido
            try:
                titulo_pt = driver.find_element(By.CSS_SELECTOR, 'span.hero__primary-text').text
            except:
                titulo_pt = ''

            # Título Original
            try:
                titulo_og = driver.find_element(By.CSS_SELECTOR, 'div.sc-ec65ba05-1.fUCCIx').text
                titulo_og = titulo_og[17:] # Removendo a parte do "Título original: "
            except:
                titulo_og = titulo_pt
            
            # Ano e Duração
            try:
                duracao_ano_bruto = driver.find_elements(By.CSS_SELECTOR, 'div.sc-70a366cc-0.bxYZmb > ul > li')
                duracao_ano = [info.text for info in duracao_ano_bruto]
                ano = duracao_ano[0]
                duracao = duracao_ano[2]
                if 'h' in duracao:
                    pos_h = duracao.find('h')
                    horas = int(duracao[:pos_h]) * 60
                    pos_min = duracao.find('m')
                    minutos = int(duracao[pos_h + 1:pos_min])
                    duracao = horas + minutos
                else:
                    pos_min = duracao.find('m')
                    duracao = int(duracao[:pos_min])
            except:
                ano = duracao = 0

            # Equipe
            try:
                ficha_tecnica = driver.find_elements(By.CSS_SELECTOR, 'div.sc-70a366cc-3.iwmAOx > div > ul > li > div > ul > li > a')
                diretor = [membro.text for membro in ficha_tecnica if '_dr_' in membro.get_attribute('href')]
                roteirista = [membro.text for membro in ficha_tecnica if '_wr_' in membro.get_attribute('href')]
                elenco = [membro.text for membro in ficha_tecnica if '_st_' in membro.get_attribute('href')]              
            except:
                diretor = roteirista = elenco = ''

            # Nota
            try:
                info_nota = driver.find_element(By.CSS_SELECTOR, 'div[class="sc-d541859f-0 hNIoIx"]').text
                pos_barra = info_nota.find('/10')
                nota = float(info_nota[:pos_barra].replace(',', '.'))
                num_votos = info_nota[pos_barra + 3:].lstrip().replace(',', '.')
                if 'mil' in num_votos:
                    pos_m = num_votos.find('m')
                    num_votos = float(num_votos[:pos_m]) * 1_000
                elif 'mi':
                    pos_m = num_votos.find('m')
                    num_votos = float(num_votos[:pos_m]) * 1_000_000
            except:
                nota = num_votos = 0

            # Sinopse
            try:
                sinopse = driver.find_element(By.CSS_SELECTOR, 'span[class="sc-3ac15c8d-1 gkeSEi"]').text
            except:
                sinopse = ''

            # Verba e Receita
            try:
                verba = driver.find_element(By.CSS_SELECTOR, 'li[data-testid="title-boxoffice-budget"]').text
                pos_dinheiro_verba = verba.find('$') + 1
                pos_estimativa = verba.find('(estimativa)')
                verba = int(verba[pos_dinheiro_verba:pos_estimativa].strip().replace('.', ''))

                receita = driver.find_element(By.CSS_SELECTOR, 'li[data-testid="title-boxoffice-cumulativeworldwidegross"]').text
                pos_dinheiro_receita = receita.find('$') + 1
                receita = int(receita[pos_dinheiro_receita:].strip().replace('.', ''))
            except:
                verba = receita = 0

            # Gênero
            try:
                elemento_scroll = driver.find_element(By.CSS_SELECTOR, 'hgroup')
                driver.execute_script("arguments[0].scrollIntoView();", elemento_scroll)
                time.sleep(2)
                generos_bruto = driver.find_elements(By.CSS_SELECTOR, 'div[class="sc-d6b15a1a-0 bfFXAE"] > ul > li > div > ul > li > a')
                generos = [genero.text for genero in generos_bruto]
            except:
                generos = ''

            # Adicionando a lista
            filme_raspado = {'titulo': titulo_pt,
                            'titulo_original': titulo_og,
                            'ano': ano,
                            'duracao': duracao,
                            'diretor': diretor,
                            'roteirista': roteirista,
                            'elenco': elenco,
                            'nota': nota,
                            'num_votos': num_votos,
                            'sinopse': sinopse,
                            'genero': generos,
                            'verba': verba,
                            'receita': receita}
            
            lista_filmes.append(filme_raspado)
        except:
            print('Erro dentro de um filme')
except:
    print('Erro ao obter URLs')
    pass

driver.quit()



# Salvando o conteúdo
colunas = lista_filmes[0].keys()

with open('../data/top_250_filmes.csv', 'w', newline='', encoding='utf-8') as arquivo:
    writer = csv.DictWriter(arquivo, fieldnames=colunas)
    writer.writeheader()
    writer.writerows(lista_filmes)

### Análises dos Usuários

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

driver = webdriver.Chrome()
driver.get('https://www.imdb.com/chart/top/')

lista_analises = []

try:
    time.sleep(2)
    urls = driver.find_elements(By.CSS_SELECTOR, 'ul > li > div > div > div > div > a')
    url_filmes = [filme.get_attribute('href') for filme in urls]

    for url in url_filmes:
        try:
            pos_url = url.find('?ref')
            url = url[:pos_url] + 'reviews'
            analises = []
            driver.get(url)
            time.sleep(2)
            
            # Título
            try:
                titulo = driver.find_element(By.CSS_SELECTOR, 'h2').text
            except:
                titulo = ''

            # Análises
            for x in range(3):
                button = driver.find_element(By.CSS_SELECTOR, 'span[class="ipc-see-more sc-33e570c-0 cMGrFN single-page-see-more-button"] > button')
                driver.execute_script("arguments[0].scrollIntoView();", button)
                time.sleep(2)
                button.click()

            [analises.append(analise.text) for analise in driver.find_elements(By.CSS_SELECTOR, 'div[class="ipc-html-content-inner-div"]')]          

            # Adicionando a lista
            filme_raspado = {'titulo': titulo,
                             'analises': analises}
            
            lista_analises.append(filme_raspado)

        except:
            pass                

except:
    print('Erro ao obter URLs')
    pass

driver.quit()



# Salvando o conteúdo
colunas = lista_analises[0].keys()

with open('../data/top_250_analises.csv', 'w', newline='', encoding='utf-8') as arquivo:
    writer = csv.DictWriter(arquivo, fieldnames=colunas)
    writer.writeheader()
    writer.writerows(lista_analises)

## Os 25 mais assistidos de cada ano

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

driver = webdriver.Chrome()

lista_filmes = []

for ano in range(1950, 2025):
    driver.get(f'https://www.imdb.com/search/title/?title_type=feature&release_date={ano}-01-01,{ano}-12-31&sort=num_votes,desc')
    try:
        time.sleep(2)
        urls = driver.find_elements(By.CSS_SELECTOR, 'ul > li > div > div > div > div > div > div > a[class="ipc-title-link-wrapper"]')
        url_filmes = [filme.get_attribute('href') for filme in urls]

        for url in url_filmes[:25]:
            driver.get(url)
            time.sleep(2)
            try:
                # Título Traduzido
                try:
                    titulo_pt = driver.find_element(By.CSS_SELECTOR, 'span.hero__primary-text').text
                except:
                    titulo_pt = ''

                # Título Original
                try:
                    titulo_og = driver.find_element(By.CSS_SELECTOR, 'div.sc-ec65ba05-1.fUCCIx').text
                    titulo_og = titulo_og[17:] # Removendo a parte do "Título original: "
                except:
                    titulo_og = titulo_pt
                
                # Ano e Duração
                try:
                    duracao_ano_bruto = driver.find_elements(By.CSS_SELECTOR, 'div.sc-70a366cc-0.bxYZmb > ul > li')
                    duracao_ano = [info.text for info in duracao_ano_bruto]
                    ano = duracao_ano[0]
                    duracao = duracao_ano[2]
                    if 'h' in duracao:
                        pos_h = duracao.find('h')
                        horas = int(duracao[:pos_h]) * 60
                        pos_min = duracao.find('m')
                        minutos = int(duracao[pos_h + 1:pos_min])
                        duracao = horas + minutos
                    else:
                        pos_min = duracao.find('m')
                        duracao = int(duracao[:pos_min])
                except:
                    ano = duracao = 0

                # Equipe
                try:
                    ficha_tecnica = driver.find_elements(By.CSS_SELECTOR, 'div.sc-70a366cc-3.iwmAOx > div > ul > li > div > ul > li > a')
                    diretor = [membro.text for membro in ficha_tecnica if '_dr_' in membro.get_attribute('href')]
                    roteirista = [membro.text for membro in ficha_tecnica if '_wr_' in membro.get_attribute('href')]
                    elenco = [membro.text for membro in ficha_tecnica if '_st_' in membro.get_attribute('href')]              
                except:
                    diretor = roteirista = elenco = ''

                # Nota
                try:
                    info_nota = driver.find_element(By.CSS_SELECTOR, 'div[class="sc-d541859f-0 hNIoIx"]').text
                    pos_barra = info_nota.find('/10')
                    nota = float(info_nota[:pos_barra].replace(',', '.'))
                    num_votos = info_nota[pos_barra + 3:].lstrip().replace(',', '.')
                    if 'mil' in num_votos:
                        pos_m = num_votos.find('m')
                        num_votos = float(num_votos[:pos_m]) * 1_000
                    elif 'mi':
                        pos_m = num_votos.find('m')
                        num_votos = float(num_votos[:pos_m]) * 1_000_000
                except:
                    nota = num_votos = 0

                # Sinopse
                try:
                    sinopse = driver.find_element(By.CSS_SELECTOR, 'span[class="sc-3ac15c8d-1 gkeSEi"]').text
                except:
                    sinopse = ''

                # Verba e Receita
                try:
                    verba = driver.find_element(By.CSS_SELECTOR, 'li[data-testid="title-boxoffice-budget"]').text
                    pos_dinheiro_verba = verba.find('$') + 1
                    pos_estimativa = verba.find('(estimativa)')
                    verba = int(verba[pos_dinheiro_verba:pos_estimativa].strip().replace('.', ''))

                    receita = driver.find_element(By.CSS_SELECTOR, 'li[data-testid="title-boxoffice-cumulativeworldwidegross"]').text
                    pos_dinheiro_receita = receita.find('$') + 1
                    receita = int(receita[pos_dinheiro_receita:].strip().replace('.', ''))
                except:
                    verba = receita = 0

                # Gênero
                try:
                    elemento_scroll = driver.find_element(By.CSS_SELECTOR, 'hgroup')
                    driver.execute_script("arguments[0].scrollIntoView();", elemento_scroll)
                    time.sleep(2)
                    generos_bruto = driver.find_elements(By.CSS_SELECTOR, 'div[class="sc-d6b15a1a-0 bfFXAE"] > ul > li > div > ul > li > a')
                    generos = [genero.text for genero in generos_bruto]
                except:
                    generos = ''

                # Análises
                try:
                    pos_url = url.find('?ref')
                    url = url[:pos_url] + 'reviews'
                    analises = []
                    driver.get(url)
                    time.sleep(2)
                    for x in range(3):
                        button = driver.find_element(By.CSS_SELECTOR, 'span[class="ipc-see-more sc-33e570c-0 cMGrFN single-page-see-more-button"] > button')
                        driver.execute_script("arguments[0].scrollIntoView();", button)
                        time.sleep(2)
                        button.click()
                    [analises.append(analise.text) for analise in driver.find_elements(By.CSS_SELECTOR, 'div[class="ipc-html-content-inner-div"]')]          
                except:
                    pass                

                # Adicionando a lista
                filme_raspado = {'titulo': titulo_pt,
                                'titulo_original': titulo_og,
                                'ano': ano,
                                'duracao': duracao,
                                'diretor': diretor,
                                'roteirista': roteirista,
                                'elenco': elenco,
                                'nota': nota,
                                'num_votos': num_votos,
                                'sinopse': sinopse,
                                'genero': generos,
                                'verba': verba,
                                'receita': receita,
                                'analises': analises}
                
                lista_filmes.append(filme_raspado)
            except:
                print('Erro dentro de um filme')
    except:
        print('Erro ao obter URLs')
        pass

colunas = lista_filmes[0].keys()

with open(f'../data/filmes.csv', 'w', newline='', encoding='utf-8') as arquivo:
    writer = csv.DictWriter(arquivo, fieldnames=colunas)
    writer.writeheader()
    writer.writerows(lista_filmes)

driver.quit()